In [ ]:
!pip install prince

In [33]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# 차원축소
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
import prince


In [34]:
data = pd.read_csv('./data_to_model/data_with_removed_outlier.csv', index_col=0)
data.fillna(0, inplace=True)
data.head(3)

,2023년09월_전체세대,2023년09월_1인세대,2023년09월_2인세대,2023년09월_3인세대,2023년09월_4인세대,2023년09월_5인세대,2023년9월_6인가구이상,채소가게수,제로웨이스트 가게수,유치원수,...,2_40대,2_50대,2_60대,2_70대,2_80대,2_90대,2_기타,카페수,승하차인원_버스_지하철,1점포당_1년_매출액
가양2동,8020,4650,1987,829,428,84,42,0.0,0.0,3.0,...,1374,1774,3360,1948,1172,226,4,12.0,90113.0,30928029.88
가양3동,7746,3743,2294,1098,481,106,24,0.0,0.0,4.0,...,1678,2008,3056,2014,1054,160,6,9.0,67861.0,53256287.08
갈현1동,7455,3614,1708,1165,764,161,43,3.0,0.0,1.0,...,1972,2670,2642,1528,732,106,4,40.0,232027.0,27594935.59


In [35]:
df = data.copy()
df.columns

Index(['2023년09월_전체세대', '2023년09월_1인세대', '2023년09월_2인세대', '2023년09월_3인세대',
       '2023년09월_4인세대', '2023년09월_5인세대', '2023년9월_6인가구이상', '채소가게수',
       '제로웨이스트 가게수', '유치원수', '거주인구대비취약자수', '거주인구대비취약자백분위수', '미세먼지지수',
       '미세먼지백분위', '행정구역면적', '공원수', '공원율', '도시농업(프로그램수)', '도시농업(시설수)', '초등학교수',
       '사업체수 (개)', '종사자수 (명)', '채식식당수', '인구수', '1_0대', '1_10대', '1_20대',
       '1_30대', '1_40대', '1_50대', '1_60대', '1_70대', '1_80대', '1_90대', '1_기타',
       '2_0대', '2_10대', '2_20대', '2_30대', '2_40대', '2_50대', '2_60대', '2_70대',
       '2_80대', '2_90대', '2_기타', '카페수', '승하차인원_버스_지하철', '1점포당_1년_매출액'],
      dtype='object')

In [36]:
# 인구 범위 축소하기
df['남자_10대이하'] = df['1_0대'] + df['1_10대']
df['남자_20_30대'] = df['1_20대'] + df['1_30대']
df['남자_40_60대'] = df['1_40대'] + df['1_50대'] + df['1_60대']
df['남자_70대이상'] = df['1_70대'] + df['1_80대'] + df['1_90대'] + df['1_기타']

df['여자_10대이하'] = df['2_0대'] + df['2_10대']
df['여자_20_30대'] = df['2_20대'] + df['2_30대']
df['여자_40_60대'] = df['2_40대'] + df['2_50대'] + df['2_60대']
df['여자_70대이상'] = df['2_70대'] + df['2_80대'] + df['2_90대'] + df['2_기타']

# 유치원, 초등학교 수 합치기
df['유치원_초등학교'] = df['유치원수'] + df['초등학교수']

In [37]:
df.columns

Index(['2023년09월_전체세대', '2023년09월_1인세대', '2023년09월_2인세대', '2023년09월_3인세대',
       '2023년09월_4인세대', '2023년09월_5인세대', '2023년9월_6인가구이상', '채소가게수',
       '제로웨이스트 가게수', '유치원수', '거주인구대비취약자수', '거주인구대비취약자백분위수', '미세먼지지수',
       '미세먼지백분위', '행정구역면적', '공원수', '공원율', '도시농업(프로그램수)', '도시농업(시설수)', '초등학교수',
       '사업체수 (개)', '종사자수 (명)', '채식식당수', '인구수', '1_0대', '1_10대', '1_20대',
       '1_30대', '1_40대', '1_50대', '1_60대', '1_70대', '1_80대', '1_90대', '1_기타',
       '2_0대', '2_10대', '2_20대', '2_30대', '2_40대', '2_50대', '2_60대', '2_70대',
       '2_80대', '2_90대', '2_기타', '카페수', '승하차인원_버스_지하철', '1점포당_1년_매출액',
       '남자_10대이하', '남자_20_30대', '남자_40_60대', '남자_70대이상', '여자_10대이하',
       '여자_20_30대', '여자_40_60대', '여자_70대이상', '유치원_초등학교'],
      dtype='object')

In [38]:
population_feature = ['2023년09월_1인세대', '남자_10대이하', '남자_20_30대', '남자_40_60대', '남자_70대이상', '여자_10대이하',
       '여자_20_30대', '여자_40_60대', '여자_70대이상']

# infra_feature = ['공원수', '도시농업(프로그램수)', '도시농업(시설수)']
# demand_feature = ['채소가게수', '카페수', '채식식당수', '초등학교수', '유치원수']
environ_featrue = ['미세먼지지수', '미세먼지백분위', '거주인구대비취약자수', '공원수', '도시농업(프로그램수)', '도시농업(시설수)']
market_feature = ['1점포당_1년_매출액', '승하차인원_버스_지하철', '종사자수 (명)', '사업체수 (개)', '채소가게수', '카페수', '채식식당수', '초등학교수', '유치원수']

In [7]:
# # 인구 합치기 전, 스케일링
# scaled_data2 = StandardScaler().fit_transform(np.log1p(data))
# data_scale_log2=pd.DataFrame(data=scaled_data2, columns=data.columns)

In [ ]:
# # 인구 합친 후, 스케일링 (로그, 스탠다드)
# scale_kind = 'standard_log'
# scaled_data = StandardScaler().fit_transform(np.log1p(df))
# data_scale_log=pd.DataFrame(data=scaled_data, columns=df.columns)
# data_scale_log.head(5)

In [39]:
# 인구 합친 후, 스케일링 (min-max)
scale_kind = 'min_max'
scaled_data = MinMaxScaler().fit_transform(df)
data_scale_log=pd.DataFrame(data=scaled_data, columns=df.columns)
data_scale_log.head()

,2023년09월_전체세대,2023년09월_1인세대,2023년09월_2인세대,2023년09월_3인세대,2023년09월_4인세대,2023년09월_5인세대,2023년9월_6인가구이상,채소가게수,제로웨이스트 가게수,유치원수,...,1점포당_1년_매출액,남자_10대이하,남자_20_30대,남자_40_60대,남자_70대이상,여자_10대이하,여자_20_30대,여자_40_60대,여자_70대이상,유치원_초등학교
0,0.452482,0.437267,0.488077,0.288359,0.147143,0.151737,0.266234,0.000,0.0,0.500000,...,0.333839,0.153462,0.270953,0.401737,0.575773,0.1444,0.361073,0.393024,0.659642,0.500
1,0.436776,0.350730,0.564332,0.384122,0.166071,0.191956,0.149351,0.000,0.0,0.666667,...,0.580707,0.188080,0.326615,0.400401,0.478066,0.1972,0.432044,0.407444,0.636581,0.625
2,0.420096,0.338422,0.418778,0.407974,0.267143,0.292505,0.272727,0.375,0.0,0.166667,...,0.296987,0.281941,0.345010,0.439279,0.428714,0.2664,0.400933,0.440843,0.464811,0.125
3,0.713516,0.449385,0.830104,0.816661,0.578929,0.696527,0.649351,0.125,0.0,0.666667,...,0.189866,0.614561,0.586052,0.812558,0.745264,0.6604,0.756562,0.848780,0.815507,0.750
4,0.142841,0.059250,0.145057,0.199359,0.190357,0.261426,0.331169,0.500,0.0,0.166667,...,0.588238,0.090650,0.048065,0.096994,0.123629,0.1036,0.064651,0.096808,0.105567,0.250


In [ ]:
# # 인구 합친 후, 스케일링 (log, min-max)
# scaled_data = MinMaxScaler().fit_transform(np.log1p(df))
# data_scale_log=pd.DataFrame(data=scaled_data, columns=df.columns)
# data_scale_log.head()

#### 인구특성



In [40]:
population_feature = ['2023년09월_1인세대', '남자_10대이하', '남자_20_30대', '남자_40_60대', '남자_70대이상', '여자_10대이하',
       '여자_20_30대', '여자_40_60대', '여자_70대이상']

pca = PCA(n_components=0.95)
population_scale_data = data_scale_log[population_feature]
pca.fit(population_scale_data)
pca_population = pd.DataFrame((pca.transform(population_scale_data)))

num_of_principal = pca_population.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 3


In [41]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=population_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.6777516145592198
제 2주성분 : 0.22068578524814728
제 3주성분 : 0.07426199825655232


,0,1,2
2023년09월_1인세대,-0.182091,0.566711,-0.249983
남자_10대이하,-0.337349,-0.467736,-0.202266
남자_20_30대,-0.278458,0.276414,-0.290460
남자_40_60대,-0.380586,-0.019393,0.161138
남자_70대이상,-0.322951,0.153501,0.454604
여자_10대이하,-0.365686,-0.504928,-0.231537
여자_20_30대,-0.352347,0.271720,-0.473812
여자_40_60대,-0.383225,-0.079336,0.137598
여자_70대이상,-0.348741,0.156975,0.531564


In [42]:
population_column = ['인구가 비교적 적은 지역', '1인 세대 및 20~30대 남녀가 많은 지역', '노령 인구가 많은 지역']

---

### 환경 관련 인프라 특성

In [43]:
environ_feature = ['미세먼지지수', '미세먼지백분위', '거주인구대비취약자수', '공원수', '도시농업(프로그램수)', '도시농업(시설수)']

pca = PCA(n_components=0.85)
environ_scale_data = data_scale_log[environ_feature]
pca.fit(environ_scale_data)
pca_environ = pd.DataFrame((pca.transform(environ_scale_data)))

num_of_principal = pca_environ.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 2


In [44]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=environ_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.5370422829517691
제 2주성분 : 0.31841280740307915


,0,1
미세먼지지수,0.610855,-0.035129
미세먼지백분위,0.788612,-0.024351
거주인구대비취약자수,-0.016358,0.657988
공원수,0.068403,0.751815
도시농업(프로그램수),0.000000,0.000000
도시농업(시설수),0.000000,0.000000


이상치 제거 후, 도시농업 데이터나 0으로 변경되어 영향력이 사라짐

In [45]:
environ_column = ['공기질 오염이 심한 지역', '공원녹지가 많은 지역']

### 상권 특성

In [48]:
market_feature = ['1점포당_1년_매출액', '승하차인원_버스_지하철', '종사자수 (명)', '사업체수 (개)', '채소가게수', '카페수', '채식식당수', '초등학교수', '유치원수']

pca = PCA(n_components=0.9)
market_scale_data = data_scale_log[market_feature]
pca.fit(market_scale_data)
pca_market = pd.DataFrame((pca.transform(market_scale_data)))

num_of_principal = pca_market.shape[1] 
print('주성분의 개수:', num_of_principal)

주성분의 개수: 6


In [49]:
print('주성분의 설명력')
for i in range(num_of_principal) :
    print(f"제 {i+1}주성분 :", pca.explained_variance_ratio_[i])
    
pd.DataFrame(data=pca.components_,columns=market_scale_data.columns).T

주성분의 설명력
제 1주성분 : 0.3129945775292557
제 2주성분 : 0.21289500232008568
제 3주성분 : 0.15547555201491134
제 4주성분 : 0.09216627205706351
제 5주성분 : 0.07895376902980764
제 6주성분 : 0.07048986461621767


,0,1,2,3,4,5
1점포당_1년_매출액,0.058219,0.075403,-0.225955,0.948146,-0.030040,-0.172878
승하차인원_버스_지하철,0.277811,0.062807,0.096376,0.057965,-0.510455,0.127554
종사자수 (명),0.397816,0.097517,-0.093017,0.060092,-0.068382,0.354772
사업체수 (개),0.447087,0.007162,0.043183,-0.009158,-0.104693,0.438115
채소가게수,0.245569,-0.462717,0.783035,0.168871,0.064006,-0.249666
카페수,0.453339,0.070470,-0.034888,-0.060616,0.020274,0.025552
채식식당수,0.537040,0.151521,-0.244605,-0.178535,0.414785,-0.554917
초등학교수,-0.083192,0.693127,0.466408,0.127121,0.443719,0.239208
유치원수,-0.011533,0.508284,0.193418,-0.117211,-0.591147,-0.455956


In [15]:
market_column = ['사업체와 카페가 많은 지역', '초등학교 및 유치원이 많은 지역', '거주 지역', '상가 매출이 많은 곳','유동인구가 많은 지역']

## 차원축소 결과

In [20]:
pca_result =pd.concat([pca_population, pca_environ, pca_market],axis=1)

In [21]:
pca_result.columns= population_column + environ_column + market_column
pca_result.index = data.index
pca_result.head()

,인구가 비교적 적은 지역,1인 세대 및 20~30대 남녀가 많은 지역,노령 인구가 많은 지역,공기질 오염이 심한 지역,공원녹지가 많은 지역,사업체가 많고 채식 소비가 많은 지역,초등학교 및 유치원이 많은 지역,거주 지역,상가 매출이 많은 곳,유동인구가 많은 지역
가양2동,0.338207,0.301216,0.303639,0.087892,0.036253,-0.348108,0.156950,-0.257588,-0.086238,-0.024465
가양3동,0.317052,0.224256,0.201343,0.075567,-0.104940,-0.360364,0.243992,-0.248914,0.173667,-0.214045
갈현1동,0.316409,0.097126,0.075928,-0.392406,0.114279,0.097132,-0.377073,-0.156749,-0.059318,-0.104315
갈현2동,-0.675440,0.032787,0.097775,-0.395642,0.244427,0.080925,0.462681,0.075103,-0.179652,-0.138109
개포1동,1.178407,-0.132153,0.035540,0.358068,-0.511077,-0.447158,-0.269525,0.068464,0.317769,0.119895


In [31]:
data_kind = 'removed_outlier_'

In [32]:
pca_result.to_csv(f'./data_to_model/data_after_pca_with_{data_kind}{scale_kind}_scaled_{len(pca_result.columns)}_features.csv')